# In this example we use Random forest classifier to predict the heart disease 

In [1]:
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql._
import org.apache.spark.ml.feature._
import org.apache.spark.ml._
import org.apache.spark.sql.types.{StructType, StructField, StringType, IntegerType,DoubleType};
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.sql._
import org.apache.spark.ml.feature.{VectorAssembler}
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.{DecisionTreeClassificationModel,DecisionTreeClassifier}
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder}
import org.apache.spark.mllib.evaluation.MulticlassMetrics

In [2]:
val conf = new SparkConf().setAppName("Predicting_Heart_Heart_Disease")
val sc = new SparkContext(conf)
val spark = new SQLContext(sc) 

conf = org.apache.spark.SparkConf@251ac71f
sc = org.apache.spark.SparkContext@5e9d7da5
spark = org.apache.spark.sql.SQLContext@7ccf327e


org.apache.spark.sql.SQLContext@7ccf327e

In [3]:
val schema = StructType(Seq(StructField("age",DoubleType,true), StructField("sex",DoubleType,true), 
               StructField("cp",DoubleType,true), 
               StructField("trestbps",DoubleType,true), StructField("chol",DoubleType,true), 
               StructField("fbs",DoubleType,true), StructField("restecg",DoubleType,true), 
               StructField("thalach",DoubleType,true), StructField("exang",DoubleType,true),
               StructField("oldpeak",DoubleType,true),StructField("slope",DoubleType,true),
               StructField("ca",DoubleType,true) ,StructField("thal",DoubleType,true),
               StructField("num",DoubleType,true)))

schema = StructType(StructField(age,DoubleType,true), StructField(sex,DoubleType,true), StructField(cp,DoubleType,true), StructField(trestbps,DoubleType,true), StructField(chol,DoubleType,true), StructField(fbs,DoubleType,true), StructField(restecg,DoubleType,true), StructField(thalach,DoubleType,true), StructField(exang,DoubleType,true), StructField(oldpeak,DoubleType,true), StructField(slope,DoubleType,true), StructField(ca,DoubleType,true), StructField(thal,DoubleType,true), StructField(num,DoubleType,true))


StructType(StructField(age,DoubleType,true), StructField(sex,DoubleType,true), StructField(cp,DoubleType,true), StructField(trestbps,DoubleType,true), StructField(chol,DoubleType,true), StructField(fbs,DoubleType,true), StructField(restecg,DoubleType,true), StructField(thalach,DoubleType,true), StructField(exang,DoubleType,true), StructField(oldpeak,DoubleType,true), StructField(slope,DoubleType,true), StructField(ca,DoubleType,true), StructField(thal,DoubleType,true), StructField(num,DoubleType,true))

In [4]:
val data = spark.read.format("csv")
                .option("header","false")
                .option("delimiter", ",")
                .schema(schema)
                .load("/user/viswatejaster9073/ML/heart-disease.csv")

data = [age: double, sex: double ... 12 more fields]


[age: double, sex: double ... 12 more fields]

In [7]:
data.show(5)

+----+---+---+--------+-----+---+-------+-------+-----+-------+-----+---+----+---+
| age|sex| cp|trestbps| chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|num|
+----+---+---+--------+-----+---+-------+-------+-----+-------+-----+---+----+---+
|63.0|1.0|1.0|   145.0|233.0|1.0|    2.0|  150.0|  0.0|    2.3|  3.0|0.0| 6.0|0.0|
|67.0|1.0|4.0|   160.0|286.0|0.0|    2.0|  108.0|  1.0|    1.5|  2.0|3.0| 3.0|2.0|
|67.0|1.0|4.0|   120.0|229.0|0.0|    2.0|  129.0|  1.0|    2.6|  2.0|2.0| 7.0|1.0|
|37.0|1.0|3.0|   130.0|250.0|0.0|    0.0|  187.0|  0.0|    3.5|  3.0|0.0| 3.0|0.0|
|41.0|0.0|2.0|   130.0|204.0|0.0|    2.0|  172.0|  0.0|    1.4|  1.0|0.0| 3.0|0.0|
+----+---+---+--------+-----+---+-------+-------+-----+-------+-----+---+----+---+
only showing top 5 rows



In [11]:
data.count()

303

In [5]:
data.map(x=>x.anyNull).filter($"value" === true).count

6

In [6]:
val f_data=data.na.drop()

f_data = [age: double, sex: double ... 12 more fields]


[age: double, sex: double ... 12 more fields]

In [7]:
f_data.map(x=>x.anyNull).filter($"value" === true).count

0

# Checking the count of the heart patents based on Sec
- In our case we have a Sex column with the value 1 and 0 as Male and Female

In [8]:
val heart_patents_count = f_data.select($"sex",$"num").groupBy("sex").sum("num")

heart_patents_count = [sex: double, sum(num): double]


[sex: double, sum(num): double]

## As per the below results we have 52 Female and 229 Male patents who are attacked with heart disease

In [9]:
heart_patents_count.show()

+---+--------+
|sex|sum(num)|
+---+--------+
|0.0|    52.0|
|1.0|   229.0|
+---+--------+



# In our case we need to predict the value of num, so lets name it as label for our conveneant 

In [7]:
val f1_data = f_data.withColumnRenamed("num","label")

f1_data = [age: double, sex: double ... 12 more fields]


[age: double, sex: double ... 12 more fields]

In [11]:
f1_data.printSchema

root
 |-- age: double (nullable = true)
 |-- sex: double (nullable = true)
 |-- cp: double (nullable = true)
 |-- trestbps: double (nullable = true)
 |-- chol: double (nullable = true)
 |-- fbs: double (nullable = true)
 |-- restecg: double (nullable = true)
 |-- thalach: double (nullable = true)
 |-- exang: double (nullable = true)
 |-- oldpeak: double (nullable = true)
 |-- slope: double (nullable = true)
 |-- ca: double (nullable = true)
 |-- thal: double (nullable = true)
 |-- label: double (nullable = true)



# Vectorize the features
- Once we make a Vector from our features we are writing the data in to a file with LIBS

In [10]:
val assembler = new VectorAssembler().setInputCols(f_data.columns.filter(!_.contains("num"))).setOutputCol("features")

assembler = vecAssembler_9cae1a8f8637


lastException: Throwable = null


vecAssembler_9cae1a8f8637

In [9]:
val output = assembler.transform(f1_data).select($"label",$"features").
                write.format("libsvm").save("/user/viswatejaster9073/ML/heart_libsvm.txt")

Name: org.apache.spark.sql.AnalysisException
Message: path hdfs://ip-172-31-35-141.ec2.internal:8020/user/viswatejaster9073/ML/heart_libsvm.txt already exists.;
StackTrace:   at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:109)
  at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
  at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
  at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
  at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
  at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
  at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
  at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
  at org.apache

## Read the file that we stored as libsvm format

In [11]:
val model_data = spark.read.format("libsvm").load("/user/viswatejaster9073/ML/heart_libsvm.txt")

model_data = [label: double, features: vector]


[label: double, features: vector]

In [10]:
model_data.printSchema

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



# Split the Data for train and test data

In [12]:
val Array(train,test1) = model_data.randomSplit(Array(0.7,0.3),seed=1234)

train = [label: double, features: vector]
test1 = [label: double, features: vector]


[label: double, features: vector]

In [34]:
train.count()

216

In [35]:
test1.count()

81

# Model using Random Forest with the help of Cross Validation

In [13]:
val nFolds: Int = 10
val NumTrees: Int = 30
val mdepth: Int = 5
val Impurity:String = "gini"

nFolds = 10
NumTrees = 30
mdepth = 5
Impurity = gini


gini

In [14]:
val rf_tree = new RandomForestClassifier()
  .setLabelCol("label")
  .setFeaturesCol("features")
  .setNumTrees(NumTrees).setMaxDepth(mdepth).setImpurity(Impurity)

rf_tree = rfc_ab1cc89d976b


rfc_ab1cc89d976b

In [15]:
val pipeline = new Pipeline().setStages(Array(rf_tree)) 

pipeline = pipeline_3da361fcc643


pipeline_3da361fcc643

### We are building a parameter Grid with mutiple combinaitons for number of trees, depth of each tree, and impurtiy 

In [16]:
val paramGrid = new ParamGridBuilder()
  .addGrid(rf_tree.maxDepth, Array(2,3,4,5,6,7,8))
  .addGrid(rf_tree.numTrees, Array(2,3,4,5,6,7,8))
  .addGrid(rf_tree.impurity, Array("entropy", "gini"))
  .build()

paramGrid = 


Array({
	rfc_ab1cc89d976b-impurity: entropy,
	rfc_ab1cc89d976b-maxDepth: 2,
	rfc_ab1cc89d976b-numTrees: 2
}, {
	rfc_ab1cc89d976b-impurity: gini,
	rfc_ab1cc89d976b-maxDepth: 2,
	rfc_ab1cc89d976b-numTrees: 2
}, {
	rfc_ab1cc89d976b-impurity: entropy,
	rfc_ab1cc89d976b-maxDepth: 2,
	rfc_ab1cc89d976b-numTrees: 3
}, {
	rfc_ab1cc89d976b-impurity: gini,
	rfc_ab1cc89d976b-maxDepth: 2,
	rfc_ab1cc89d976b-numTrees: 3
}, {
	rfc_ab1cc89d976b-impurity: entropy,
	rfc_ab1cc89d976b-maxDepth: 2,
	rfc_ab1cc89d976b-numTrees: 4
}, {
	rfc_ab1cc89d976b-impurity: gini,
	rfc_ab1cc89d976b-maxDepth: 2,
	rfc_ab1cc89d976b-numTrees: 4
}, {
	rfc_ab1cc89d976b-impurity: entropy,
	rfc_ab1cc89d976b-maxDepth: 2,
	rfc_ab1cc89d976b-numTrees: 5
}, {
	rfc_ab1cc89d976b-impu...


[{
	rfc_ab1cc89d976b-impurity: entropy,
	rfc_ab1cc89d976b-maxDepth: 2,
	rfc_ab1cc89d976b-numTrees: 2
}, {
	rfc_ab1cc89d976b-impurity: gini,
	rfc_ab1cc89d976b-maxDepth: 2,
	rfc_ab1cc89d976b-numTrees: 2
}, {
	rfc_ab1cc89d976b-impurity: entropy,
	rfc_ab1cc89d976b-maxDepth: 2,
	rfc_ab1cc89d976b-numTrees: 3
}, {
	rfc_ab1cc89d976b-impurity: gini,
	rfc_ab1cc89d976b-maxDepth: 2,
	rfc_ab1cc89d976b-numTrees: 3
}, {
	rfc_ab1cc89d976b-impurity: entropy,
	rfc_ab1cc89d976b-maxDepth: 2,
	rfc_ab1cc89d976b-numTrees: 4
}, {
	rfc_ab1cc89d976b-impurity: gini,
	rfc_ab1cc89d976b-maxDepth: 2,
	rfc_ab1cc89d976b-numTrees: 4
}, {
	rfc_ab1cc89d976b-impurity: entropy,
	rfc_ab1cc89d976b-maxDepth: 2,
	rfc_ab1cc89d976b-numTrees: 5
}, {
	rfc_ab1cc89d976b-impurity: gini,
	rfc_ab1cc89d976b-maxDepth: 2,
	rfc_ab1cc89d976b-numTrees: 5
}, {
	rfc_ab1cc89d976b-impurity: entropy,
	rfc_ab1cc89d976b-maxDepth: 2,
	rfc_ab1cc89d976b-numTrees: 6
}, {
	rfc_ab1cc89d976b-impurity: gini,
	rfc_ab1cc89d976b-maxDepth: 2,
	rfc_ab1cc89d976b

## Building a cross Validation Model with the help of the Parameter grid we created above
- The lenght of the parameter Grid is 98 which means our Cross validation will create 98 different Random Forest and will chose the best Random Forest based on the accurecy

In [18]:
paramGrid.length

98

### We need to mention on what basis the best Random Forest need to be selected out of 98 Random Forest
- We can choose Accuracy,Precision,Recall based on our requrement 

In [19]:
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("label")
  .setMetricName("accuracy")

evaluator = mcEval_59feb9b7b2dd


mcEval_59feb9b7b2dd

### Building Corss Validation with K fold 10

In [20]:
val cv = new CrossValidator()
  .setEstimator(pipeline)
  .setEvaluator(evaluator)
  .setEstimatorParamMaps(paramGrid)
  .setNumFolds(10)

cv = cv_3e5b8201d466


cv_3e5b8201d466

### Fitting Our Model with Train Data

In [1]:
val r_model = cv.fit(train)

r_model = cv_3e5b8201d466


cv_3e5b8201d466

### We are testing our model with the test Data

In [2]:
val re = r_model.transform(test1)

re = [label: double, features: vector ... 3 more fields]


[label: double, features: vector ... 3 more fields]

### We are testing Our model performance

In [3]:
def model_accurecy(metric: String): Double = {
    
    val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName(metric)
    
  evaluator.evaluate(re)   
}

model_accurecy: (metric: String)Double


In [4]:
model_accurecy("accuracy")

0.5802469135802469

In [5]:
r_model.write.overwrite.save("/user/viswatejaster9073/ML1/r_forest_hear")